# Generate summed flag count tables

This notebook creates QAQC flag counts csv files per network from the corresponding eraqc_counts_timestep files that were generated as a part of the final processing step for stations within the Historical Data Pipeline. These tables are used to then generate statistics for the QAQC success report.

This is carried out in two steps:

1. Generate the per-network QAQC flag count tables, at native and hourly timesteps

2. Generates one flag count table that sums all per-network tables, at native and hourly timesteps


Using the following functions:


- _pairwise_sum(): helper function that merges two input flag tables, used by network_sum_flag_counts() and total_sum_flag_counts().

- network_sum_flag_counts(): sums all station flag count tables for a given network, creating one flag count table for that network

- generate_station_tables(): runs network_sum_flag_counts() for every network

- total_sum_flag_counts(): sums all network flag count tables, creating one final flag count table 

## Step 0: Environment set-up

In [44]:
import time
import boto3
import numpy as np
import pandas as pd
import xarray as xr

from qaqc_success_report_functions import *

## Step 1: Generate flag sum tables for ever network

First, loop through every network, combining each of their station flag count tables into one table. The result is one flag count table at each timestep - native and hourly - for every network.

This will take around 1 hour to run for both timesteps. 

In [ ]:
generate_station_tables("native")

In [45]:
generate_station_tables('hourly')

Sending summed counts dataframe for ASOSAWOS to: s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/ASOSAWOS_flag_counts_hourly_timestep.csv
Sending summed counts dataframe for CAHYDRO to: s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/CAHYDRO_flag_counts_hourly_timestep.csv
Sending summed counts dataframe for CDEC to: s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/CDEC_flag_counts_hourly_timestep.csv
Sending summed counts dataframe for CIMIS to: s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/CIMIS_flag_counts_hourly_timestep.csv
Sending summed counts dataframe for CNRFC to: s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/CNRFC_flag_counts_hourly_timestep.csv
Sending summed counts dataframe for CRN to: s3://wecc-historical-wx/4_merge_wx/per_network_flag_counts_hourly_timestep/CRN_flag_counts_hourly_timestep.csv
Sending summed counts dataframe for CW3E t

## Step 2: Generate total flag sum table

Now combine all the network flag count tables generated in step 1 into one final flag count table. First at the hourly timestep, and then at the native timestep.

Step 1 must be complete before moving on to this step.

In [ ]:
total_sum_flag_counts('native')

In [ ]:
total_sum_flag_counts('hourly')